In [0]:
dbutils.widgets.text("catalog_use", "fhir_workshop")
dbutils.widgets.text("schema_use", spark.sql("SELECT REPLACE(SPLIT(current_user(), '@')[0], '.', '_')").collect()[0][0])

In [0]:
catalog_use = dbutils.widgets.get("catalog_use")
schema_use = dbutils.widgets.get("schema_use")

In [0]:
spark.sql(f"USE {catalog_use}.{schema_use}")

In [0]:
%sql
SELECT current_catalog(), current_schema();

In [0]:
resource_types = spark.sql("SELECT DISTINCT resourceType FROM resource_schemas").collect()
resource_types = [row.resourceType for row in resource_types]
resource_types

In [0]:
from pyspark.sql.functions import col

In [0]:
resource_control = []
for resource_type in resource_types:
  resource_dict = {}
  resource_dict["resource_type"] = resource_type
  current_columns = (
    spark.sql(f"DESCRIBE {resource_type}")
    .filter(~col("col_name").isin("bundle_uuid", f"{resource_type}_url"))
    .select("col_name")
  )
  latest_columns = spark.sql(f"SELECT resource from resource_schemas where resourceType = '{resource_type}'")
  columns_diff = latest_columns.select("resource").subtract(current_columns.select(col("col_name").alias("resource")))

  if columns_diff.count() > 0:
      resource_dict["full_refresh"] = "true"
  else:
      resource_dict["full_refresh"] = "false"
  
  resource_control.append(resource_dict)

In [0]:
resource_control

In [0]:
dbutils.jobs.taskValues.set("resource_control", resource_control)